# Distance Model Integration
BS"D

In this notebook, I will explore how to integrate the filtering model and the similar ingredients model to create the distance model.

The API will accept a list of ingredients (a recipe) and a set of dietary restrictions. It will return a new recipe that meets the dietary restrictions and is similar to the original recipe.

In [2]:
import pandas as pd
import torch.nn as nn
from collections import OrderedDict

from filtering_model.filtering_model import FilteringModel
from similar_ingredients.get_similar_ingredients import load_model as load_similar_ingredients_model

/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the Models
### Load the Filtering Model
This will be based on how the filtering model is loaded in the "Data Filtering" notebook.

It seems like the following steps are needed to initialize the filtering model:
1. Load the training data
2. Define the column name with the ingredient names
3. Specify the embeddings to use
4. Build the internal model

After this the model must be trained and then it can be used to filter the data.

In [2]:
# Load the training data
file_path = "../../data_preparation/classification_dataset/common_ingredients.csv"
training_data = pd.read_csv(file_path)

training_data

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,salt,yes,yes,yes,yes
1,olive oil,yes,yes,yes,yes
2,onions,yes,yes,yes,yes
3,water,yes,yes,yes,yes
4,garlic,yes,yes,yes,yes
...,...,...,...,...,...
494,boneless chicken breast,no,no,yes,yes
495,crème fraîche,yes,no,no,yes
496,cooked white rice,yes,yes,yes,yes
497,pecans,yes,yes,yes,yes


In [4]:
# Build the internal model
internal_model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(768, 256)),
    ('relu1', nn.LeakyReLU()),
    ('bn1', nn.BatchNorm1d(256)),
    ('fc2', nn.Linear(256, 64)),
    ('dr1', nn.Dropout(0.3)),
    ('relu2', nn.LeakyReLU()),
    ('bn2', nn.BatchNorm1d(64)),
    ('fc3', nn.Linear(64, 4)),
    ('sg1', nn.Sigmoid())
]))

In [5]:
ingredient_column = 'ingredient'
embedding_model = "facebook/drama-base"

filtering_model = FilteringModel(training_data, ingredient_column, embedding_model, internal_model)

In [6]:
# Train the model
filtering_model.train_model(epochs=20,batch_size=33,val_split=0.2)

/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 1 | Train Loss: 8.351 | Val Loss: 2.812 | Train Acc: 0.163 | Val Acc: 0.03 | Train Pre: 0.992 | Val Pre: 0.7 | Train Rec: 0.992 | Val Rec: 0.7
Epoch: 2 | Train Loss: 7.199 | Val Loss: 2.595 | Train Acc: 0.27 | Val Acc: 0.023 | Train Pre: 0.994 | Val Pre: 0.727 | Train Rec: 0.994 | Val Rec: 0.727
Epoch: 3 | Train Loss: 6.646 | Val Loss: 2.226 | Train Acc: 0.396 | Val Acc: 0.182 | Train Pre: 0.998 | Val Pre: 0.931 | Train Rec: 0.998 | Val Rec: 0.931
Epoch: 4 | Train Loss: 6.17 | Val Loss: 1.762 | Train Acc: 0.51 | Val Acc: 0.705 | Train Pre: 0.998 | Val Pre: 0.987 | Train Rec: 0.998 | Val Rec: 0.987
Epoch: 5 | Train Loss: 5.956 | Val Loss: 1.794 | Train Acc: 0.585 | Val Acc: 0.652 | Train Pre: 0.998 | Val Pre: 0.99 | Train Rec: 0.998 | Val Rec: 0.99
Epoch: 6 | Train Loss: 5.157 | Val Loss: 1.515 | Train Acc: 0.667 | Val Acc: 0.795 | Train Pre: 0.998 | Val Pre: 0.984 | Train Rec: 0.998 | Val Rec: 0.984
Epoch: 7 | Train Loss: 4.798 | Val Loss: 1.431 | Train Acc: 0.695 | Val Acc: 0.8

/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 9 | Train Loss: 4.256 | Val Loss: 1.53 | Train Acc: 0.762 | Val Acc: 0.735 | Train Pre: 0.998 | Val Pre: 0.986 | Train Rec: 0.998 | Val Rec: 0.986
Epoch: 10 | Train Loss: 3.989 | Val Loss: 2.024 | Train Acc: 0.795 | Val Acc: 0.53 | Train Pre: 0.997 | Val Pre: 0.864 | Train Rec: 0.997 | Val Rec: 0.864
Epoch: 11 | Train Loss: 3.648 | Val Loss: 1.769 | Train Acc: 0.802 | Val Acc: 0.614 | Train Pre: 0.99 | Val Pre: 0.802 | Train Rec: 0.99 | Val Rec: 0.802


/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 12 | Train Loss: 3.006 | Val Loss: 0.884 | Train Acc: 0.86 | Val Acc: 0.871 | Train Pre: 0.999 | Val Pre: 0.981 | Train Rec: 0.999 | Val Rec: 0.981
Epoch: 13 | Train Loss: 2.906 | Val Loss: 0.838 | Train Acc: 0.844 | Val Acc: 0.871 | Train Pre: 0.995 | Val Pre: 0.918 | Train Rec: 0.995 | Val Rec: 0.918
Epoch: 14 | Train Loss: 2.507 | Val Loss: 1.078 | Train Acc: 0.9 | Val Acc: 0.864 | Train Pre: 0.995 | Val Pre: 0.987 | Train Rec: 0.995 | Val Rec: 0.987
Epoch: 15 | Train Loss: 2.238 | Val Loss: 1.131 | Train Acc: 0.925 | Val Acc: 0.644 | Train Pre: 0.996 | Val Pre: 0.911 | Train Rec: 0.996 | Val Rec: 0.911
Epoch: 16 | Train Loss: 2.064 | Val Loss: 0.894 | Train Acc: 0.872 | Val Acc: 0.886 | Train Pre: 0.995 | Val Pre: 0.979 | Train Rec: 0.995 | Val Rec: 0.979
Epoch: 17 | Train Loss: 1.684 | Val Loss: 0.595 | Train Acc: 0.942 | Val Acc: 0.886 | Train Pre: 0.998 | Val Pre: 0.972 | Train Rec: 0.998 | Val Rec: 0.972
Epoch: 18 | Train Loss: 1.485 | Val Loss: 0.552 | Train Acc: 0.96 |

In [8]:
# Try it out on a hamburger recipe
recipe = ["beef", "onion", "garlic", "salt", "pepper", "cheese", "lettuce", "tomato", "bun"]

filtering_model.filter(recipe, threshold=0.5)

,ingredient,vegetarian,vegan,dairy_free,gluten_free
0,beef,no,no,yes,yes
1,onion,yes,yes,yes,yes
2,garlic,yes,yes,yes,yes
3,salt,yes,yes,yes,yes
4,pepper,yes,yes,yes,yes
5,cheese,yes,no,yes,yes
6,lettuce,yes,yes,yes,yes
7,tomato,yes,yes,yes,yes
8,bun,yes,yes,yes,yes


#### Loading the Filtering Model V2.0

In [11]:
filtering_model_2 = FilteringModel(training_data)

In [12]:
filtering_model_2.train_model()

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256])

### Load the Similar Ingredients Model

In [3]:
file_path = "similar_ingredients/all_ingredients.json"

similar_ingredients_model = load_similar_ingredients_model(file_path)

In [15]:
# Get similar ingredients for each ingredient in the recipe
similar_ingredients = {}

for ingredient in recipe:
    # Get similar ingredients
    similar = similar_ingredients_model(ingredient)
    
    # Store the similar ingredients in the dictionary
    similar_ingredients[ingredient] = similar

# Add the results to a dataframe
similar_ingredients_df = pd.DataFrame.from_dict(similar_ingredients, orient='index').reset_index()

similar_ingredients_df

,index,0,1,2,3,4,5,6,7,8,9
0,beef,"(meat, 0.7286377549171448)","(Beef, 0.7230441570281982)","(pork, 0.6601365208625793)","(veal, 0.6273409724235535)","(lamb, 0.5826153755187988)","(chicken, 0.577904999256134)","(Meat, 0.5627167820930481)","(chuck roast, 0.5397898554801941)","(Pork, 0.5307552814483643)","(grassfed beef, 0.5286645293235779)"
1,onion,"(onions, 0.8558207750320435)","(cauliflower, 0.702358603477478)","(bell pepper, 0.6711967587471008)","(potatoes, 0.6605713963508606)","(garlic, 0.6516684293746948)","(asparagus, 0.6511366963386536)","(shallots, 0.6489439606666565)","(tomatoes, 0.6406427621841431)","(eggplant, 0.6400142908096313)","(cabbage, 0.6383322477340698)"
2,garlic,"(fennel, 0.7304399609565735)","(oregano, 0.7030937075614929)","(parsley, 0.7022287249565125)","(basil, 0.6971802711486816)","(cilantro, 0.6914703249931335)","(onions, 0.6913739442825317)","(shallots, 0.6845468878746033)","(cabbage, 0.6810933947563171)","(rosemary, 0.680901288986206)","(roasted garlic, 0.6715270280838013)"
3,salt,"(Salt, 0.589695155620575)","(brine, 0.5670109391212463)","(garlic salt, 0.5598366856575012)","(kosher salt, 0.5527639985084534)","(Kosher salt, 0.5492516756057739)","(coarse kosher salt, 0.544251561164856)","(coarse salt, 0.5065730810165405)","(beet juice, 0.4978453814983368)","(wasabi powder, 0.48599207401275635)","(unsalted butter, 0.4835364818572998)"
4,pepper,"(cumin, 0.6287866830825806)","(capsicum, 0.6136705875396729)","(garlic, 0.607893705368042)","(peppercorns, 0.604446291923523)","(coriander, 0.6012625694274902)","(sesame seeds, 0.5909296870231628)","(paprika, 0.5800351500511169)","(mustard seeds, 0.5795686841011047)","(cinnamon, 0.576282262802124)","(pepper flakes, 0.575015127658844)"
5,cheese,"(goat cheese, 0.7297402620315552)","(Cheese, 0.7286962270736694)","(cheddar cheese, 0.725513756275177)","(Cheddar cheese, 0.6943709254264832)","(Camembert, 0.6623162627220154)","(mozzarella cheese, 0.6550924777984619)","(havarti, 0.6534529328346252)","(camembert, 0.6466249227523804)","(Mozzarella cheese, 0.6437987685203552)","(ricotta, 0.6399509310722351)"
6,lettuce,"(spinach, 0.7631042003631592)","(tomatoes, 0.7614434957504272)","(iceberg lettuce, 0.7238020896911621)","(broccoli, 0.715250551700592)","(leaf lettuce, 0.7119097709655762)","(romaine lettuce, 0.7091070413589478)","(cantaloupe, 0.688593864440918)","(asparagus, 0.6758716106414795)","(celery, 0.6734380722045898)","(cauliflower, 0.6686928272247314)"
7,tomato,"(tomatoes, 0.8442263007164001)","(lettuce, 0.7069937586784363)","(asparagus, 0.7050933837890625)","(peaches, 0.6938520073890686)","(cherry tomatoes, 0.6897530555725098)","(strawberries, 0.6832595467567444)","(cantaloupe, 0.6780219078063965)","(celery, 0.675195574760437)","(spinach, 0.6682621240615845)","(cauliflower, 0.668158769607544)"
8,bun,"(buns, 0.7008675932884216)","(ciabatta roll, 0.5964027643203735)","(toasted buns, 0.5916598439216614)","(ciabatta, 0.5904300808906555)","(shredded cheddar cheese, 0.5894902944564819)","(mashed potatoes, 0.5863385200500488)","(french bread, 0.585828423500061)","(breadstick, 0.5819747447967529)","(cutlet, 0.5814052224159241)","(baguette, 0.5781702995300293)"


In [10]:
similar_ingredients_model("beef", top_n=1)

Time taken to find similar ingredients: 0.12674307823181152 seconds


[('meat', 0.7286377549171448)]

## Integrate the Models
The final model will have three steps:
1. Flag the ingredients that violate the dietary restrictions (using the filtering model)
2. Find similar ingredients for the flagged ingredients (using the similar ingredients model)
3. Pick a new ingredient from the similar ingredients list that is not flagged (using the filtering model)

I'd like to define a few hyperparameters for the model that control the threshold for the filtering model for step 1 and step 3, and the number of similar ingredients to return for step 2.

In [66]:
threshold_1 = 0.5
threshold_2 = 0.5
num_similar_ingredients = 20

In [71]:
# Step 1: Flag the ingredients that violate the dietary restrictions
def flag_violations(recipe, dietary_restrictions, threshold=0.5):
    '''
    Flag ingredients in the recipe that violate dietary restrictions.

    Parameters
    -----------
    recipe : list
        List of ingredients in the recipe.

    dietary_restrictions : list
        List of dietary restrictions to check against. (Can be "vegan", "vegetarian", "gluten_free", or "dairy_free")

    Returns
    --------
    non_violations : list
        List of ingredients that do not violate the dietary restrictions.

    violations : list
        List of ingredients that violate the dietary restrictions.
    '''

    # First run the recipe through the filtering model
    filtered_recipe = filtering_model.filter(recipe, threshold=threshold)

    # Then convert it to a dictionary of dictionaries
    recipe_dict = filtered_recipe.to_dict(orient='records')

    # Split the recipe into non-violations and violations
    non_violations = []
    violations = []

    for ingredient in recipe_dict:
        violation = False

        for restriction in dietary_restrictions:
            if ingredient[restriction] == "no":
                violation = True
                break

        if violation:
            violations.append(ingredient['ingredient'])
        else:
            non_violations.append(ingredient['ingredient'])
    return non_violations, violations

In [72]:
restrictions = ["vegan", "gluten_free"]

non_violations, violations = flag_violations(recipe, restrictions, threshold=threshold_1)

print("Non-violations:", ", ".join(non_violations))
print("Violations:", ", ".join(violations))

Non-violations: onion, garlic, salt, pepper, lettuce, tomato
Violations: beef, cheese, bun


In [73]:
# Step 2: Get similar ingredients for the violations
def get_similar_ingredients(violations, top_n = num_similar_ingredients):
    '''
    Get similar ingredients for the violations.

    Parameters
    -----------
    violations : list
        List of ingredients that violate the dietary restrictions.

    top_n : int
        Number of similar ingredients to return.

    Returns
    --------
    similar_ingredients : dict
        Dictionary of ingredients and their similar ingredients.
    '''

    # Get similar ingredients for each violation
    similar_ingredients = {}

    for ingredient in violations:
        # Get similar ingredients
        similar = similar_ingredients_model(ingredient, top_n=top_n)
    
        # Store the similar ingredients in the dictionary
        similar_ingredients[ingredient] = similar

    return similar_ingredients

In [74]:
similar_ingredients = get_similar_ingredients(violations)

similar_ingredients

{'beef': [('meat', 0.7286377549171448),
  ('Beef', 0.7230441570281982),
  ('pork', 0.6601365208625793),
  ('veal', 0.6273409724235535),
  ('lamb', 0.5826153755187988),
  ('chicken', 0.577904999256134),
  ('Meat', 0.5627167820930481),
  ('chuck roast', 0.5397898554801941),
  ('Pork', 0.5307552814483643),
  ('grassfed beef', 0.5286645293235779),
  ('pork sausages', 0.5261837840080261),
  ('venison', 0.5257208347320557),
  ('sirloin', 0.5230457186698914),
  ('spinach', 0.5227606296539307),
  ('beef sirloin', 0.5160597562789917),
  ('poultry', 0.5132147073745728),
  ('steak', 0.5128119587898254),
  ('hamburger', 0.5067266225814819),
  ('cheese', 0.5016943216323853),
  ('cheddar cheese', 0.5011695027351379)],
 'cheese': [('goat cheese', 0.7297402620315552),
  ('Cheese', 0.7286962270736694),
  ('cheddar cheese', 0.725513756275177),
  ('Cheddar cheese', 0.6943709254264832),
  ('Camembert', 0.6623162627220154),
  ('mozzarella cheese', 0.6550924777984619),
  ('havarti', 0.6534529328346252),
  (

In [76]:
# Step 3: Pick ingredients from the similar ingredients list that do not violate the dietary restrictions
def pick_substitutes(similar_ingredients, dietary_restrictions, num_suggestions = 3):
    '''
    Pick ingredients from the similar ingredients list that do not violate the dietary restrictions.

    Parameters
    -----------
    similar_ingredients : dict
        Dictionary of ingredients and their similar ingredients.

    dietary_restrictions : list
        List of dietary restrictions to check against. (Can be "vegan", "vegetarian", "gluten_free", or "dairy_free")

    num_suggestions : int
        Number of suggestions to return for each violation.

    Returns
    --------
    substitutes : dict
        Dictionary of violations and their substitutes.
    '''

    # Get substitutes for each violation
    substitutes = {}
    
    for ingredient, similar in similar_ingredients.items():
        # Convert the similar ingredients to a dictionary from the ingredient to the similarity score
        similar_dict = {i[0]: i[1] for i in similar}

        # Get a list of the similar ingredients
        similar_list = list(similar_dict.keys())

        # Check which of them violate the dietary restrictions
        non_violations, _ = flag_violations(similar_list, dietary_restrictions, threshold=threshold_2)

        # Sort the non-violations by their similarity score
        non_violations = sorted(non_violations, key=lambda x: similar_dict[x], reverse=True)

        # Pick the top n suggestions
        substitutes[ingredient] = non_violations[:num_suggestions]
    return substitutes

In [77]:
picked_substitutes = pick_substitutes(similar_ingredients, restrictions, num_suggestions=5)
picked_substitutes

{'beef': ['veal', 'spinach'],
 'cheese': ['havarti', 'Reblochon', 'fontina', 'Fontina'],
 'bun': ['ciabatta roll',
  'toasted buns',
  'ciabatta',
  'mashed potatoes',
  'cutlet']}

### Combine the Steps

In [93]:
def generate_substitutes(recipe, dietary_restrictions, num_suggestions=3):
    '''
    Generate substitutes for the recipe based on the dietary restrictions.

    Parameters
    -----------
    recipe : list
        List of ingredients in the recipe.

    dietary_restrictions : list
        List of dietary restrictions to check against. (Can be "vegan", "vegetarian", "gluten_free", or "dairy_free")

    num_suggestions : int
        Number of suggestions to return for each violation.

    Returns
    --------
    substitutes : dict
        Dictionary of violations and their substitutes.
    '''

    # Flag the violations
    non_violations, violations = flag_violations(recipe, dietary_restrictions, threshold=threshold_1)

    # Get similar ingredients for the violations
    similar_ingredients = get_similar_ingredients(violations, top_n=num_similar_ingredients)

    # Pick substitutes from the similar ingredients list that do not violate the dietary restrictions
    substitutes = pick_substitutes(similar_ingredients, dietary_restrictions, num_suggestions=num_suggestions)

    return non_violations, substitutes

In [100]:
threshold_1 = 0.5
threshold_2 = 0.9
num_similar_ingredients = 100

generate_substitutes(recipe, restrictions, num_suggestions=5)

(['onion', 'garlic', 'salt', 'pepper', 'lettuce', 'tomato'],
 {'beef': ['spinach',
   'potatoes',
   'fresh spinach',
   'Russet potatoes',
   'asparagus'],
  'cheese': ['Taleggio',
   'grana padano',
   'spicy salami',
   'robiola',
   'pepperoni slices'],
  'bun': ['mashed potatoes',
   'vermicelli noodles',
   'Mashed potatoes',
   'lavash',
   'corn muffin']})

## Encapsulate the Model
The final model will be encapsulated in a class that has the following methods:
1. `__init__`: Initialize the model, and get it ready for use
2. generate_substitutes: This will take a list of ingredients and dietary restrictions, and return a new recipe that meets the dietary restrictions and is similar to the original recipe.
3. set_hyperparameters: This will set the hyperparameters for the model.
4. get_hyperparameters: This will return the hyperparameters for the model.

### Demonstration
First, the models package must be added to the python path.

In [11]:
# Add the root of the project to the path
# This file is integration_exploration.ipynb
import os
import sys
from pathlib import Path

# Get the root directory of the project
root_dir = Path(os.getcwd()).resolve().parent.parent
# Add the root of the project to the path
sys.path.append(str(root_dir))

In [12]:
# Import the integrated model
from models.distance_model.integrated_model import DistanceModel

In [15]:
model = DistanceModel(
    filtering_model_training_data_path='../../data_preparation/classification_dataset/common_ingredients.csv',
    similar_ingredients_all_ingredients_path='similar_ingredients/all_ingredients.json'
)

/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 1 | Train Loss: 8.579 | Val Loss: 2.968 | Train Acc: 0.152 | Val Acc: 0.0 | Train Pre: 0.955 | Val Pre: 0.43 | Train Rec: 0.955 | Val Rec: 0.43
Epoch: 2 | Train Loss: 7.518 | Val Loss: 2.754 | Train Acc: 0.284 | Val Acc: 0.053 | Train Pre: 0.996 | Val Pre: 0.742 | Train Rec: 0.996 | Val Rec: 0.742
Epoch: 3 | Train Loss: 7.091 | Val Loss: 2.216 | Train Acc: 0.401 | Val Acc: 0.576 | Train Pre: 0.999 | Val Pre: 0.992 | Train Rec: 0.999 | Val Rec: 0.992
Epoch: 4 | Train Loss: 6.69 | Val Loss: 1.901 | Train Acc: 0.469 | Val Acc: 0.667 | Train Pre: 0.998 | Val Pre: 0.994 | Train Rec: 0.998 | Val Rec: 0.994
Epoch: 5 | Train Loss: 5.969 | Val Loss: 1.725 | Train Acc: 0.545 | Val Acc: 0.818 | Train Pre: 0.999 | Val Pre: 0.993 | Train Rec: 0.999 | Val Rec: 0.993


/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 6 | Train Loss: 5.723 | Val Loss: 1.627 | Train Acc: 0.625 | Val Acc: 0.508 | Train Pre: 0.998 | Val Pre: 0.804 | Train Rec: 0.998 | Val Rec: 0.804
Epoch: 7 | Train Loss: 5.17 | Val Loss: 1.725 | Train Acc: 0.683 | Val Acc: 0.75 | Train Pre: 0.99 | Val Pre: 0.928 | Train Rec: 0.99 | Val Rec: 0.928
Epoch: 8 | Train Loss: 4.944 | Val Loss: 1.452 | Train Acc: 0.727 | Val Acc: 0.856 | Train Pre: 0.999 | Val Pre: 0.99 | Train Rec: 0.999 | Val Rec: 0.99
Epoch: 9 | Train Loss: 4.484 | Val Loss: 1.284 | Train Acc: 0.776 | Val Acc: 0.864 | Train Pre: 0.999 | Val Pre: 0.995 | Train Rec: 0.999 | Val Rec: 0.995
Epoch: 10 | Train Loss: 4.297 | Val Loss: 3.311 | Train Acc: 0.772 | Val Acc: 0.591 | Train Pre: 0.999 | Val Pre: 0.863 | Train Rec: 0.999 | Val Rec: 0.863


/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 11 | Train Loss: 3.859 | Val Loss: 1.323 | Train Acc: 0.818 | Val Acc: 0.591 | Train Pre: 0.998 | Val Pre: 0.799 | Train Rec: 0.998 | Val Rec: 0.799
Epoch: 12 | Train Loss: 3.296 | Val Loss: 1.656 | Train Acc: 0.846 | Val Acc: 0.644 | Train Pre: 0.999 | Val Pre: 0.863 | Train Rec: 0.999 | Val Rec: 0.863
Epoch: 13 | Train Loss: 2.976 | Val Loss: 0.938 | Train Acc: 0.893 | Val Acc: 0.879 | Train Pre: 0.999 | Val Pre: 0.984 | Train Rec: 0.999 | Val Rec: 0.984
Epoch: 14 | Train Loss: 2.687 | Val Loss: 1.664 | Train Acc: 0.916 | Val Acc: 0.652 | Train Pre: 0.999 | Val Pre: 0.914 | Train Rec: 0.999 | Val Rec: 0.914


/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

Epoch: 15 | Train Loss: 2.564 | Val Loss: 0.749 | Train Acc: 0.9 | Val Acc: 0.917 | Train Pre: 0.998 | Val Pre: 0.922 | Train Rec: 0.998 | Val Rec: 0.922
Epoch: 16 | Train Loss: 2.061 | Val Loss: 2.139 | Train Acc: 0.925 | Val Acc: 0.674 | Train Pre: 0.996 | Val Pre: 0.86 | Train Rec: 0.996 | Val Rec: 0.86
Epoch: 17 | Train Loss: 1.989 | Val Loss: 0.666 | Train Acc: 0.937 | Val Acc: 0.917 | Train Pre: 0.997 | Val Pre: 0.98 | Train Rec: 0.997 | Val Rec: 0.98


/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tuvyamacklin/Documents/Repos/Ingredient-Substitution-Capstone/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch: 18 | Train Loss: 1.779 | Val Loss: 1.717 | Train Acc: 0.93 | Val Acc: 0.659 | Train Pre: 0.996 | Val Pre: 0.862 | Train Rec: 0.996 | Val Rec: 0.862
Epoch: 19 | Train Loss: 1.513 | Val Loss: 0.583 | Train Acc: 0.953 | Val Acc: 0.894 | Train Pre: 0.997 | Val Pre: 0.974 | Train Rec: 0.997 | Val Rec: 0.974
Epoch: 20 | Train Loss: 1.392 | Val Loss: 0.537 | Train Acc: 0.974 | Val Acc: 0.894 | Train Pre: 1.0 | Val Pre: 0.968 | Train Rec: 1.0 | Val Rec: 0.968


In [16]:
recipe = ["beef", "onion", "garlic", "salt", "pepper", "cheese", "lettuce", "tomato", "hamburger bun"]
restrictions = ["vegan"]

In [17]:
model.generate_substitutes(recipe, restrictions)

['veal',
 'onion',
 'garlic',
 'salt',
 'pepper',
 'havarti',
 'lettuce',
 'tomato',
 'honey mustard dressing']

## Handling Missing Words
_4/6/2025_

One of the main errors that we are encountering when using the full model is that the similar ingredients model does not have embeddings for many ingredients. This mainly occurs when the ingredient is a multi-word ingredient. Here I will explore a solution to this problem.

My idea is to create an embedding which is the average of the embeddings of the individual words. Then search for that vector in the model.

I will also make the model more robust by handling the case where the model can't find an embedding for the input at all.

**Update**: Actually it looks like the feature to concatenate multi-word ingredients was never implemented. I will implement it now.

In [1]:
# Load the get_similar_ingredients model

from similar_ingredients.get_similar_ingredients import load_model as load_similar_ingredients_model

file_path = "similar_ingredients/all_ingredients.json"
similar_ingredients_model = load_similar_ingredients_model(file_path)

In [2]:
similar_ingredients_model("bun")

[('buns', 0.7008675932884216),
 ('ciabatta roll', 0.5964027643203735),
 ('toasted buns', 0.5916598439216614),
 ('ciabatta', 0.5904300808906555),
 ('shredded cheddar cheese', 0.5894902944564819),
 ('mashed potatoes', 0.5863385200500488),
 ('french bread', 0.585828423500061),
 ('breadstick', 0.5819747447967529),
 ('cutlet', 0.5814052224159241),
 ('baguette', 0.5781702995300293)]

In [3]:
similar_ingredients_model("hamburger bun")

[('honey mustard dressing', 0.6502435207366943),
 ('french bread', 0.645281195640564),
 ('nacho chips', 0.6402700543403625),
 ('toasted baguette', 0.631892204284668),
 ('muffin', 0.6315452456474304),
 ('shredded cheddar cheese', 0.6302399039268494),
 ('alfredo sauce', 0.6273813247680664),
 ('elbow macaroni', 0.6258562803268433),
 ('sliced cucumber', 0.623972475528717),
 ('wonton skins', 0.6229598522186279)]

In [16]:
# Here is the model as it was before
import json
from gensim import downloader
all_ingredients = []
with open(file_path, "r") as f:
    all_ingredients = json.load(f)

embedding_model = downloader.load("word2vec-google-news-300")


def model(ingredient, top_n = 10):
    '''
    This function takes in an ingredient and returns a list of similar ingredients.

    Parameters
    ----------
    ingredient : str
        The ingredient for which similar ingredients are to be found.

    top_n : int
        The number of similar ingredients to return. The default value is 10.

    Returns
    -------
    list
        A list of similar ingredients.
    '''

    # Increase the top_n to compensate for the filtering
    top_n = top_n * 10

    # If the ingredient is a multi-word ingredient, replace spaces with underscores
    if ' ' in ingredient:
        ingredient = ingredient.replace(' ', '_')

    suggestions = embedding_model.most_similar(ingredient, topn=top_n)

    filtered_suggestions = []

    for suggestion, score in suggestions:
        if suggestion.lower().replace(' ', '_') in all_ingredients:
            filtered_suggestions.append((suggestion, score))

    # Remove the underscores from the suggestions
    filtered_suggestions = [(suggestion.replace('_', ' '), score) for suggestion, score in filtered_suggestions]

    # Return the top_n suggestions
    top_n = top_n // 10
    return filtered_suggestions[:top_n]

In [18]:
embedding_model.most_similar("hamburger bun")

KeyError: "Key 'hamburger bun' not present in vocabulary"

In [19]:
embedding_model.most_similar("hamburger_bun")

[('hamburger_patty', 0.6944166421890259),
 ('honey_mustard_dressing', 0.6502435207366943),
 ('bleu_cheese_dressing', 0.6498620510101318),
 ('crouton', 0.6495686173439026),
 ('breaded_fried', 0.6475617289543152),
 ('pita_pocket', 0.6464435458183289),
 ('french_bread', 0.645281195640564),
 ('tortilla_shell', 0.6448255181312561),
 ('hoagie_roll', 0.6435864567756653),
 ('pastry_crust', 0.6433483958244324)]

In [20]:
model("hamburger bun")

[('honey mustard dressing', 0.6502435207366943),
 ('french bread', 0.645281195640564),
 ('nacho chips', 0.6402700543403625),
 ('toasted baguette', 0.631892204284668),
 ('muffin', 0.6315452456474304),
 ('shredded cheddar cheese', 0.6302399039268494),
 ('alfredo sauce', 0.6273813247680664),
 ('elbow macaroni', 0.6258562803268433),
 ('sliced cucumber', 0.623972475528717),
 ('wonton skins', 0.6229598522186279)]